In [72]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from numpy import savetxt
import cv2

from PIL import Image
import pandas as pd

In [2]:
train = pd.read_csv("sign_mnist_train.csv")
test = pd.read_csv("sign_mnist_test.csv")

In [3]:
print("Shape of Train: " + str(train.shape))
print("Shape of Test: " + str(test.shape))

Shape of Train: (27455, 785)
Shape of Test: (7172, 785)


In [ ]:
# we can see that the dataframe for both train and test have ground truth labels inside, so lets separate the X and y
train

In [4]:
X_train = train.drop(columns="label").to_numpy()
y_train = train['label'].to_numpy()
X_test = test.drop(columns="label").to_numpy()
y_test = test['label'].to_numpy()

In [81]:
# if you want to save the images and labels, uncomment the codes and run everything
# https://stackoverflow.com/questions/57103190/how-to-convert-1d-array-to-3d-array-convert-grayscale-image-so-rgb-format

def resize_img(array):
    img_2d = np.reshape(array, (28, 28))
    width, height = img_2d.shape
    img = np.empty((width, height, 3), dtype=np.uint8)
    img[:, :, 0] = img_2d
    img[:, :, 1] = img_2d
    img[:, :, 2] = img_2d
    
    pil_img = tf.keras.preprocessing.image.array_to_img(img)
    return pil_img

# image_path_train = "images/train"
# image_path_test = "images/test"

train_images = []
test_images = []

for i in range(len(X_train)):
    img = resize_img(X_train[i])
    train_images.append(img)
#     img.save(f"{image_path_train}/{i}.png")

for j in range(len(X_test)):
    img = resize_img(X_test[j])
    test_images.append(img)
#     img.save(f"{image_path_test}/{j}.png")

# savetxt('labels/train_labels.csv', y_train, delimiter=',')
# savetxt('labels/test_labels.csv', y_test, delimiter=',')

(32, 32, 3)

In [70]:
# we can see that there are 24 unique labels in the dataset and each label represents a letter from A to Z (excl J and Z)
print(train['label'].nunique())
train["label"].unique()

24


array([ 3,  6,  2, 13, 16,  8, 22, 18, 10, 20, 17, 19, 21, 23, 24,  1, 12,
       11, 15,  4,  0,  5,  7, 14], dtype=int64)

In [101]:
alphabets = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E',
    5: 'F',
    6: 'G',
    7: 'H',
    8: 'I',
    9: 'K',
    10: 'L',
    11: 'M',
    12: 'N',
    13: 'O',
    14: 'P',
    15: 'Q',
    16: 'R',
    17: 'S',
    18: 'T',
    19: 'U',
    20: 'V',
    21: 'W',
    22: 'X',
    23: 'Y',
}

In [93]:
# we will define a function to resize the image to the given dimensions as different pretrained models here 
# would have different dimension requirements. We will also scale down the pixel values in this function

def resize_images(minSize, train_size, test_size):
    new_train = []
    new_test = []
    for i in range(train_size):
        img = cv2.imread('images/train/' + str(i) + '.png')
        res = cv2.resize(img, dsize=(minSize, minSize), interpolation=cv2.INTER_CUBIC)
        new_train.append(res)
        
    for j in range(test_size):
        img = cv2.imread('images/test/' + str(j) + '.png')
        res = cv2.resize(img, dsize=(minSize, minSize), interpolation=cv2.INTER_CUBIC)
        new_test.append(res)
    
    return np.array(new_train) / 255, np.array(new_test) / 255

In [94]:
train_len = 27455
test_len = 7172

X_train_scaled, X_test_scaled = resize_images(28, train_len, test_len)

In [97]:
cnn = models.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(), 

  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(25, activation='softmax')
])

cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
858/858 [==============================] - 8s 9ms/step - loss: 1.5551 - accuracy: 0.5353
Epoch 2/5
858/858 [==============================] - 8s 9ms/step - loss: 0.0204 - accuracy: 0.9962
Epoch 3/5
858/858 [==============================] - 8s 9ms/step - loss: 0.0332 - accuracy: 0.9907
Epoch 4/5
858/858 [==============================] - 8s 9ms/step - loss: 2.0239e-04 - accuracy: 1.0000
Epoch 5/5
858/858 [==============================] - 8s 9ms/step - loss: 9.5006e-05 - accuracy: 1.0000


In [98]:
cnn.evaluate(X_test_scaled, y_test)

225/225 [==============================] - 1s 3ms/step - loss: 0.2533 - accuracy: 0.9527


[0.25328513979911804, 0.9527328610420227]

In [99]:
predictions = cnn.predict(X_test_scaled)

In [102]:
score = np.argmax(tf.nn.softmax(predictions[0])) # convert into a probability score and get the highest
alphabets[score]

'G'

In [103]:
# loading pretrained models
mobilenet_v2 = tf.keras.applications.MobileNetV2(input_shape=(32,32,3), include_top=False, weights='imagenet')
vgg19 = tf.keras.applications.VGG19(input_shape=(32,32,3), include_top=False, weights='imagenet')
inceptionv3 = tf.keras.applications.InceptionV3(input_shape=(75,75,3), include_top=False, weights='imagenet')
resnet50 = tf.keras.applications.ResNet50(input_shape=(32,32,3), include_top=False, weights='imagenet')
efficientnet = tf.keras.applications.EfficientNetB1(input_shape=(32,32,3), include_top=False, weights='imagenet')


# efficientnet has B0 to B7 - need to find out the difference

In [ ]:
mobilenet_v2.summary()

In [104]:
mobilenet_v2.trainable = False

In [105]:
train_len = 27455
test_len = 7172
X_train_scaled_mobilenet, X_test_scaled_mobilenet = resize_images(32, train_len, test_len)

mobilenet_model = models.Sequential([
  mobilenet_v2,

  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(25, activation='softmax')
])

mobilenet_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

mobilenet_model.fit(X_train_scaled_mobilenet, y_train, epochs=5)

Epoch 1/5
858/858 [==============================] - 15s 15ms/step - loss: 2.5010 - accuracy: 0.3009
Epoch 2/5
858/858 [==============================] - 12s 14ms/step - loss: 1.5360 - accuracy: 0.5449
Epoch 3/5
858/858 [==============================] - 12s 15ms/step - loss: 1.2849 - accuracy: 0.6179
Epoch 4/5
858/858 [==============================] - 12s 14ms/step - loss: 1.1223 - accuracy: 0.6729
Epoch 5/5
858/858 [==============================] - 12s 14ms/step - loss: 0.9928 - accuracy: 0.7128


In [107]:
mobilenet_model.evaluate(X_test_scaled_mobilenet, y_test)

225/225 [==============================] - 4s 17ms/step - loss: 1.9218 - accuracy: 0.4727


[1.9217798709869385, 0.4726715087890625]

In [108]:
inceptionv3.trainable = False

In [109]:
train_len = 27455
test_len = 7172
X_train_scaled_inception, X_test_scaled_inception = resize_images(75, train_len, test_len)

mobilenet_model = models.Sequential([
  inceptionv3,

  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(25, activation='softmax')
])

mobilenet_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

mobilenet_model.fit(X_train_scaled_inception, y_train, epochs=5)

Epoch 1/5
858/858 [==============================] - 94s 93ms/step - loss: 0.6202 - accuracy: 0.8352
Epoch 2/5
858/858 [==============================] - 85s 99ms/step - loss: 0.0126 - accuracy: 0.9984
Epoch 3/5
858/858 [==============================] - 85s 99ms/step - loss: 0.0192 - accuracy: 0.9956
Epoch 4/5
858/858 [==============================] - 66s 77ms/step - loss: 0.0044 - accuracy: 0.9990
Epoch 5/5
858/858 [==============================] - 63s 74ms/step - loss: 0.0037 - accuracy: 0.9994


In [110]:
mobilenet_model.evaluate(X_test_scaled_inception, y_test)

225/225 [==============================] - 18s 72ms/step - loss: 0.7914 - accuracy: 0.8482


[0.7914420962333679, 0.8481594920158386]